In [1]:
import pandas as pd
import requests
from io import StringIO
from pathlib import Path

# URLs for different climate data files
data_urls = {
    "AIRPRESSURE": "https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_260_pg.txt",
    "PRECIPITATION": "https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_260_rh24.txt",
    "MINIMUM_TEMPERATURES": "https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_260_tng.txt",
    "MAXIMUM_TEMPERATURES": "https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_260_txg.txt",
    "MEAN_TEMPERATURES": "https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_260_tg.txt"
}

# Expected columns based on the structure you provided
columns = ["STN", "YYYY", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC", "YEAR"]

# Function to download and convert data to DataFrame
def load_knmi_data(url, data_type):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to download data from {url}, status code: {response.status_code}")
    
    # Read content as a text stream
    data = StringIO(response.text)
    
    # Read the file line-by-line to identify where the data starts
    lines = data.readlines()
    
    # Find the line where actual data starts (typically the line starting with 'STN')
    start_index = next(i for i, line in enumerate(lines) if line.startswith("STN,"))
    
    # Convert the lines after the header to a DataFrame
    data_str = ''.join(lines[start_index:])
    data = StringIO(data_str)
    
    try:
        # Load the data using comma as delimiter, handling missing values, and specifying columns
        df = pd.read_csv(data, sep=',', names=columns, na_values=[" "], skiprows=1, engine='python')
        df['type'] = data_type.lower()  # Add type column with data type name in lowercase
    except pd.errors.ParserError as e:
        print(f"Parser error for {url}: {e}")
        return None
    
    return df

# Dictionary to store DataFrames for each data type
dataframes = {}

# Load each dataset into a DataFrame and add it to the dictionary
for data_type, url in data_urls.items():
    df = load_knmi_data(url, data_type)
    if df is not None:
        dataframes[data_type] = df

# Function to combine all DataFrames into a single DataFrame
def combine_dataframes(dataframes):
    return pd.concat(dataframes.values(), ignore_index=True)

# Function to filter data by year range
def filter_years(df, start_year=2008, end_year=2022):
    return df[(df['YYYY'] >= start_year) & (df['YYYY'] <= end_year)]

# Function to convert monthly data to quarterly data
def convert_to_quarters(df):
    # Make a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Ensure all monthly columns are numeric, converting non-numeric values to NaN
    monthly_columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    df.loc[:, monthly_columns] = df[monthly_columns].apply(pd.to_numeric, errors='coerce')
    
    # Create new columns for each quarter by averaging the corresponding months
    df['Q1'] = df[['JAN', 'FEB', 'MAR']].mean(axis=1)
    df['Q2'] = df[['APR', 'MAY', 'JUN']].mean(axis=1)
    df['Q3'] = df[['JUL', 'AUG', 'SEP']].mean(axis=1)
    df['Q4'] = df[['OCT', 'NOV', 'DEC']].mean(axis=1)
    
    # Drop only the monthly columns, keeping quarterly data and other columns
    df = df.drop(columns=monthly_columns)
    
    return df

# Updated function to reshape to wide format with each type as a separate column
def reshape_to_wide_format(df):
    # Drop the 'STN' column as requested
    df = df.drop(columns=['STN'])
    
    # Convert to long format with 'Year_Quarter' and 'value' columns
    df_long = pd.melt(df, id_vars=['YYYY', 'type'], value_vars=['Q1', 'Q2', 'Q3', 'Q4'], 
                      var_name='Quarter', value_name='value')
    
    # Combine 'YYYY' and 'Quarter' to create 'Year_Quarter' format
    df_long['Year_Quarter'] = df_long['YYYY'].astype(str) + '-' + df_long['Quarter']
    
    # Drop the now redundant 'YYYY' and 'Quarter' columns
    df_long = df_long.drop(columns=['YYYY', 'Quarter'])
    
    # Pivot the DataFrame so each type becomes a separate column
    df_wide = df_long.pivot(index='Year_Quarter', columns='type', values='value').reset_index()
    
    # Sort by 'Year_Quarter'
    df_wide = df_wide.sort_values(by='Year_Quarter').reset_index(drop=True)
    
    return df_wide

# Function to export DataFrame to CSV
def export_to_csv(df, filename="knmi_data.csv", folder="data"):
    # Ensure the directory exists
    output_dir = Path(folder)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Define the file path
    file_path = output_dir / filename
    
    # Export to CSV
    df.to_csv(file_path, index=False)
    print(f"Data exported to {file_path}")


In [2]:
# Combine all DataFrames into one
combined_df = combine_dataframes(dataframes)
print(combined_df.shape)
combined_df.head()

(620, 16)


,STN,YYYY,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,YEAR,type
0,260,1901,,,,,,,,,,,,,,airpressure
1,260,1902,10196,10131,10108,10147,10130,10138,10163,10138,10187,10160,10163,10182,10154,airpressure
2,260,1903,10183,10212,10137,10100,10133,10167,10139,10125,10181,10069,10165,10114,10144,airpressure
3,260,1904,10173,10037,10157,10142,10156,10175,10182,10171,10196,10199,10167,10147,10159,airpressure
4,260,1905,10251,10208,10097,10114,10193,10144,10177,10133,10152,10143,10084,10257,10163,airpressure


In [3]:
# Filter the combined DataFrame for the years 2008 to 2022
filtered_df = filter_years(combined_df)
filtered_df.head()

,STN,YYYY,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,YEAR,type
107,260,2008,10140,10239,10050,10111,10159,10159,10141,10117,10178,10148,10125,10182,10146,airpressure
108,260,2009,10137,10140,10140,10141,10188,10168,10128,10165,10208,10166,10041,10068,10141,airpressure
109,260,2010,10151,10034,10168,10194,10158,10166,10158,10129,10147,10135,10057,10133,10136,airpressure
110,260,2011,10182,10161,10227,10192,10182,10154,10123,10130,10149,10182,10194,10099,10165,airpressure
111,260,2012,10194,10286,10271,10053,10159,10127,10139,10155,10149,10109,10108,10085,10153,airpressure


In [4]:
# Convert the filtered data to quarters
quarterly_df = convert_to_quarters(filtered_df)
quarterly_df.head()

,STN,YYYY,YEAR,type,Q1,Q2,Q3,Q4
107,260,2008,10146,airpressure,10143.0,10143.0,10145.333333,10151.666667
108,260,2009,10141,airpressure,10139.0,10165.666667,10167.0,10091.666667
109,260,2010,10136,airpressure,10117.666667,10172.666667,10144.666667,10108.333333
110,260,2011,10165,airpressure,10190.0,10176.0,10134.0,10158.333333
111,260,2012,10153,airpressure,10250.333333,10113.0,10147.666667,10100.666667


In [5]:
final_df = reshape_to_wide_format(quarterly_df)
final_df.head()

type,Year_Quarter,airpressure,maximum_temperatures,mean_temperatures,minimum_temperatures,precipitation
0,2008-Q1,10143.0,91.333333,58.333333,23.666667,758.0
1,2008-Q2,10143.0,188.666667,137.0,80.0,354.333333
2,2008-Q3,10145.333333,209.0,163.666667,119.333333,1135.0
3,2008-Q4,10151.666667,94.0,64.666667,32.333333,687.666667
4,2009-Q1,10139.0,65.333333,34.666667,3.333333,523.333333


In [10]:
# Function to export DataFrame to CSV
def export_to_csv(df, filename="knmi_data.csv", folder=r"C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\knmi_data"):
    """
    Export a DataFrame to a CSV file in the specified folder.
    
    Parameters:
        df (DataFrame): The DataFrame to export.
        filename (str): The name of the output CSV file.
        folder (str): The directory to save the file.
    """
    # Ensure the directory exists
    output_dir = Path(folder)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Define the file path
    file_path = output_dir / filename
    
    # Export to CSV
    df.to_csv(file_path, index=False)
    print(f"Data exported to {file_path}")


In [12]:
folder=r"C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\knmi_data"

In [13]:
# Combine all DataFrames into one
combined_df = combine_dataframes(dataframes)
print(combined_df.shape)
combined_df.head()

# Filter the combined DataFrame for the years 2008 to 2022
filtered_df = filter_years(combined_df)
filtered_df.head()

# Convert the filtered data to quarters
quarterly_df = convert_to_quarters(filtered_df)
quarterly_df.head()

# Reshape the quarterly data to wide format
final_df = reshape_to_wide_format(quarterly_df)
final_df.head()

# Export the final DataFrame to the `knmi_data` folder with the name `knmi_data.csv`
export_to_csv(final_df)


(620, 16)
Data exported to C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\knmi_data\knmi_data.csv
